In [5]:
!pip install opencv-python-headless pandas

import cv2
import numpy as np
import pandas as pd
import os




In [27]:


# Upload the video file if not already present
from google.colab import files
uploaded = files.upload()

# Move the uploaded file to the desired path
for filename in uploaded.keys():
    !mv {filename} /content/

# Verify the video file is in the content directory
!ls /content/

class Segments:
    def __init__(self, angle=0):
        self.flags = []
        self.angle = angle
        self.segments = self.create_segments()

    def create_segments(self):
        segments = [
            [[0.25, 0.8], [0.025, 0.075]],
            [[0.2, 0.75], [0.45, 0.5]],
            [[0.15, 0.75], [0.85, 0.95]],
            [[0.15, 0.35], [0.05, 0.4]],
            [[0.125, 0.325], [0.55, .85]],
            [[0.7, 0.9], [0.05, 0.4]],
            [[0.65, 0.85], [0.5, .8]]
        ]
        angle_rad = np.deg2rad(self.angle)
        cos_angle = np.cos(angle_rad)
        sin_angle = np.sin(angle_rad)
        adjusted_segments = []

        for seg in segments:
            adjusted_seg = [
                [
                    seg[0][0] * cos_angle - seg[1][0] * sin_angle,
                    seg[0][1] * cos_angle - seg[1][1] * sin_angle
                ],
                [
                    seg[0][0] * sin_angle + seg[1][0] * cos_angle,
                    seg[0][1] * sin_angle + seg[1][1] * cos_angle
                ]
            ]
            adjusted_segments.append(adjusted_seg)

        return adjusted_segments

    def digest(self, number):
        self.flags = []
        h, w = number.shape[:2]
        for a in range(len(self.segments)):
            seg = self.segments[a]
            xl, xh = seg[0]
            yl, yh = seg[1]
            xl = int(xl * w)
            xh = int(xh * w)
            yl = int(yl * h)
            yh = int(yh * h)
            sw = xh - xl
            sh = yh - yl
            if sw == 0 or sh == 0:
                continue
            count = np.count_nonzero(number[yl:yh, xl:xh] == 255)
            if count / (sh * sw) > 0.3:
                self.flags.append(a)
            print(f"Segment {a}: Count = {count}, Area = {sh * sw}, Ratio = {count / (sh * sw):.2f}")
            cv2.rectangle(number, (xl, yl), (xh, yh), (128, 128, 128), 1)

    def getNum(self):
        print(f"Flags: {self.flags}")
        if self.flags == [0, 2, 3, 4, 5, 6]:
            return 0
        if self.flags == [5, 6]:
            return 1
        if self.flags == [0, 1, 2, 4, 5]:
            return 2
        if self.flags == [0, 1, 2, 5, 6]:
            return 3
        if self.flags == [1, 3, 5, 6]:
            return 4
        if self.flags == [0, 1, 2, 3, 6]:
            return 5
        if self.flags == [0, 1, 2, 3, 4, 6]:
            return 6
        if self.flags == [0, 5, 6]:
            return 7
        if self.flags == [0, 1, 2, 3, 4, 5, 6]:
            return 8
        if self.flags == [0, 1, 2, 3, 5, 6]:
            return 9
        return -1

def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def process_video(video_path, output_csv, output_dir):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    frame_list = []
    model = Segments(angle=5)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Processing video: {video_path}")
    print(f"Total frames: {frame_count}, Frame rate: {frame_rate}")

    for i in range(0, frame_count, 6):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            print(f"Failed to read frame at position {i}")
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply binary thresholding with a manual threshold value
        threshold_value = 100  # Adjust this value as needed
        _, binary = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)

        height, width = binary.shape
        segment_width = width // 4
        segments = [binary[:, j*segment_width:(j+1)*segment_width] for j in range(4)]
        detected_numbers = []

        for j, segment in enumerate(segments):
            segment_copy = segment.copy()
            model.digest(segment_copy)
            number = model.getNum()
            detected_numbers.append(number if number != -1 else '')

            # Annotate the segment with the detected number
            cv2.putText(segment_copy, str(number), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            # Save the annotated segment
            annotated_segment_path = os.path.join(output_dir, f"frame_{i:04d}_segment_{j}.png")
            print(f"Saving segment {j} of frame {i} to {annotated_segment_path}")
            cv2.imwrite(annotated_segment_path, segment_copy)

        combined_result = ''.join(map(str, detected_numbers))
        frame_list.append(combined_result)
        print(f"Frame {i}: Detected Numbers = {combined_result}")

    cap.release()
    cv2.destroyAllWindows()

    if frame_list:
        df = pd.DataFrame(frame_list, columns=['Detected Numbers'])
        df.to_csv(output_csv, index=False)
        print(f"Output saved to {output_csv}")
    else:
        print("No frames were processed.")

# Example paths (ensure the video file exists in your working directory)
video_path = f'{filename}'
output_csv = f'{video_path}_output_data.csv'
output_dir = f'{video_path}_processed_segments/'

process_video(video_path, output_csv, output_dir)


Streaming output truncated to the last 5000 lines.
Segment 6: Count = 1, Area = 1610, Ratio = 0.00
Flags: [0, 1, 2, 4, 5]
Saving segment 3 of frame 276 to IMG_3013.mov_processed_segments/frame_0276_segment_3.png
Frame 276: Detected Numbers = 702
Segment 0: Count = 0, Area = 1606, Ratio = 0.00
Segment 1: Count = 0, Area = 1606, Ratio = 0.00
Segment 2: Count = 0, Area = 2686, Ratio = 0.00
Segment 3: Count = 0, Area = 1886, Ratio = 0.00
Segment 4: Count = 0, Area = 1610, Ratio = 0.00
Segment 5: Count = 0, Area = 1863, Ratio = 0.00
Segment 6: Count = 0, Area = 1610, Ratio = 0.00
Flags: []
Saving segment 0 of frame 282 to IMG_3013.mov_processed_segments/frame_0282_segment_0.png
Segment 0: Count = 1141, Area = 1606, Ratio = 0.71
Segment 1: Count = 0, Area = 1606, Ratio = 0.00
Segment 2: Count = 30, Area = 2686, Ratio = 0.01
Segment 3: Count = 16, Area = 1886, Ratio = 0.01
Segment 4: Count = 0, Area = 1610, Ratio = 0.00
Segment 5: Count = 1454, Area = 1863, Ratio = 0.78
Segment 6: Count = 121